In [1]:
#import sympy
import numpy as np
import copy
import matplotlib.pyplot as plt
from numpy import pi,linspace,tensordot
import scipy
import tensorflow as tf
import Spectrum as spect
import time
import tqdm
#from scipy.interpolate import interp1d

# Упрощенный расчет 0-pi

In [3]:
def upr(Ec,Ek,El,Em,dim,z,ng,r):
    print('Ec=',Ec)
    print('Ek=',Ek)
    print('El=',El)
    print('Em=',Em)
    a=-np.pi/2
    b=np.pi*3/2
    (h,b)=spect.dividing(1,[a],[b],[1],dim)
    (fi,N)=spect.basis(1,dim,[a],b)
    m=z.shape[0]
    p=ng.shape[0]
    B=np.zeros((r,m,p))
    fm2=np.tensordot(linspace(0,0,N),tensordot(linspace(0,0,r),tensordot(linspace(0,0,m),linspace(0,0,p),axes=0),axes=0),axes=0)
    A0=np.tensordot(linspace(0,0,N),linspace(0,0,N),axes=0)
    A0=spect.fillingA(Ec,A0,dim,h,mode='n**2',bound='periodic')
    for k in tqdm.tqdm(range(0,m)):
        F=z[k]*2*pi
        A1=copy.copy(A0)
        for i in range(0,N):
            A1[i,i]+=-(Ek-El*np.cos(F))*np.cos(2*fi[i,0])-Em*np.cos(F/2)*np.cos(fi[i,0])
        for l in (range(0,p)):
            A=copy.copy(A1)
            #A0=spect.fillingA(-2*Ec*ng[l],A0,dim,h,mode='n',bound='periodic')
            #for i in range(0,N):
            #    A[i,i]+=Ec*ng[l]**2   
            (B2,f)= scipy.sparse.linalg.eigsh(scipy.sparse.csr_matrix(A),k=r,which='SA',maxiter=4000)
            #(B2,f)= np.linalg.eig(A)
            l_order=np.argsort(np.real(B2))
            B2=B2[l_order]
            f=f[:,l_order]
            B[:,k,l]=B2
            fm2[:,:,k,l]=f
            #print(A)
    return (B,fm2)

In [4]:
z=np.linspace(0,1,51)
Ec=4*1.75*1e-4*40
Ek=1.8608*1e-2*40
El=1.0073*1e-8*40
Em=2.6625*1e-5*40
dim=np.asarray([1000])
(B2,fm)=upr(4*1.75*1e-4*40,1.8608*1e-2*40,1.0073*1e-8*40,2.6625*1e-5*40,dim,z,np.linspace(0,0,1),50)

  0%|                                                                                           | 0/51 [00:00<?, ?it/s]

Ec= 0.028
Ek= 0.74432
El= 4.0292e-07
Em= 0.0010650000000000002


100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:23<00:00,  2.21it/s]


In [352]:
plt.figure()
plt.plot(z,B2[0,:,0]-B2[0,:,0])
plt.plot(z,B2[1,:,0]-B2[0,:,0])

In [10]:
start=time.monotonic()
nq=50#dim[0]
m=z.shape[0]
a=-np.pi/2
b=np.pi*3/2
(h,b)=spect.dividing(1,[a],[b],[1],dim)
(fi,N)=spect.basis(1,dim,[a],b)
A0=np.tensordot(linspace(0j,0j,N),linspace(0,0,N),axes=0)
n=spect.fillingA(1,A0,dim,h,mode='n',axes=0,bound='periodic')
#n=np.diag(fi[:,0]+0j)
n_new=tf.zeros((nq,nq,m),dtype=tf.complex64)
n_new=tf.einsum('imk,il,ljk->mjk',fm[:,:nq,:,0],n,fm[:,:nq,:,0].conjugate())
#n_new=n
print(time.monotonic()-start)

8.609999999999957


In [361]:
n=n_new[:,:,0]

In [338]:
plt.figure()
for i in range(0,1):
    plt.plot(abs(fm[:,i+1,0,0]))
    #plt.plot(abs(tf.linalg.expm(-1j*n*np.pi/2)@fm[:,i:i+1,0,0]))

In [27]:
# think about writting of interaction hamiltoninian (there is an exponenta depending on time)
def evolution(nq,H1,V1,dt,step,w):
    start=time.monotonic()
    # density matrix
    #ro=np.zeros((nq,nq),dtype=complex)
    #ro[0,0]=1
    psi0=np.zeros((nq,1),dtype=complex)
    psi0[0]=1
    #psi0[1]=0
    #psi0=fm[:,0:1,0,0]
    psi=[]
    with tf.device("GPU:0"):
        for j in tqdm.tqdm(range(0,w.shape[0])):
            V=V1*w[j]
            psi1=[]
            psi1.append(psi0)
            for i in range(0,step):
                a=(tf.linalg.expm(1j*(H1+V)*dt/w[j])@psi1[i])
                psi1.append(a/np.linalg.norm(a))
                
                #psi1.append(tf.linalg.expm(1j*(H1+V)*dt/w[j])@psi1[i])
                
                #psi1.append(np.linalg.norm(tf.linalg.expm(1j*(H1+V)*dt*i)@psi0))
                #psi1.append(tf.linalg.expm(1j*(H1+V)*dt*i)@psi0)
                #psi1.append(tf.linalg.expm(-1j*(V)*2*np.pi)@tf.linalg.expm(1j*(H1)*dt*i)@tf.linalg.expm(1j*(V)*np.pi)@psi0)
                #psi1.append(tf.linalg.expm(-1j*(V)*np.pi/2)@tf.linalg.expm(1j*(H1)*dt*i)@tf.linalg.expm(1j*(V)*np.pi/2)@psi0)
            psi.append(np.asarray(psi1))
        print(time.monotonic()-start)
    return np.asarray(psi)

In [28]:
w=np.linspace(20,20,1)
step=3000
dt=0.001
nq=20
(psi1)=evolution(nq,B2[:nq,0,0]-B2[0,0,0],n_new[:nq,:nq,0],dt,step,w)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:18<00:00, 78.09s/it]

78.10899999999947


In [387]:
psi1.shape

(20, 301, 50, 1)

In [49]:
step=10
t=np.linspace(dt,step*dt,step)
plt.figure()
for i in range(0,1):
    plt.plot(t,np.abs(psi[0,:step])**2)

In [240]:
step=2
t=np.linspace(dt,step*dt,step)
plt.figure()
for i in range(0,10):
    plt.plot(t,np.abs(np.einsum('i,ji->j',fm[:,i,0,0].conjugate(),psi1[0,:step,:,0]))**2)

In [31]:
step1=step+1
t=np.linspace(dt,step1*dt,step1)
plt.figure()
for i in range(0,10):
    plt.plot(t,np.abs(psi1[0,:step1,i,0])**2)

In [30]:
step1=step+1
t=np.linspace(dt,step1*dt,step1)
plt.figure()
plt.pcolormesh(t,w,np.abs(psi1[:,:,0,0])**2)

C:\Users\mazho\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:4: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  after removing the cwd from sys.path.


In [129]:
plt.figure()
for i in range(0,2):
    plt.plot(abs(fm[:,i,0,0]))

# Другой расчет гейта 

In [3]:
def zero_pi(degrees,El1,Ej,Ec1,Ec0,E0,g1,g0,dim,period,a,b,z,ng,r):
    print('El1=',El1)
    print('Ej=',Ej)
    print('Ec1=',Ec1)
    print('Ec0=',Ec0)
    print('E0=',E0)
    print('g1=',g1)
    print('g0=',g0)
    (h,b)=spect.dividing(degrees,a,b,period,dim)
    (fi,N)=spect.basis(degrees,dim,a,b)
    m=z.shape[0]
    p=ng.shape[0]
    B=np.zeros((r,m,p))
    fm2=np.tensordot(linspace(0,0,N),tensordot(linspace(0,0,r),tensordot(linspace(0,0,m),linspace(0,0,p),axes=0),axes=0),axes=0)
    A0=np.tensordot(linspace(0,0,N),linspace(0,0,N),axes=0)
    #A0=spect.fillingA(Ec0,A0,dim,h,bound='periodic')
    A0=spect.fillingA(Ec1,A0,dim,h,axes=1)
    if degrees==3:    
        A0=spect.fillingA(g0,A0,dim,h,mode='n*nr',axesr=2)
        A0=spect.fillingA(g1,A0,dim,h,mode='n*nr',axes=1,axesr=2)
    if degrees==3:
        for i in range(0,N):
            A0[i,i]+=El1*fi[i,1]**2+E0*(fi[i,2]+0.5)
    else:
        for i in range(0,N):
            A0[i,i]+=El1*fi[i,1]**2+Ec0*fi[i,0]**2+2*Ej
    for k in range(0,m):
        F=z[k]*2*pi
        #A=A0+diag(Ej*(1-cos(0.5*fi[:,0]-0.5*fi[:,1]+F)))
        A1=copy.copy(A0)
        '''for i in range(0,N):
            A1[i,i]+=2*Ej*(1-np.cos(fi[i,0])*np.cos(fi[i,1]-F/2))'''
        for i in range(0,dim[0]-1):
            for j in range(0,dim[1]):
                A1[i*dim[1]+j,(i+1)*dim[1]+j]+=-Ej*np.cos(fi[j,1]-F/2)
                A1[(i+1)*dim[1]+j,i*dim[1]+j]+=-Ej*np.cos(fi[j,1]-F/2)
        for l in tqdm.tqdm(range(0,p)):
            A=copy.copy(A1)
            for i in range(0,N):
                A[i,i]+=2*Ec0*ng[l]*fi[i,0]+Ec0*ng[l]**2   
            (B2,f)= scipy.sparse.linalg.eigsh(scipy.sparse.csr_matrix(A),k=r,which='SA',maxiter=4000)
            l_order=np.argsort(np.real(B2))
            B2=B2[l_order]
            f=f[:,l_order]
            B[:,k,l]=B2
            fm2[:,:,k,l]=f
            #print(A)
    return (B,fm2)

In [29]:
start=time.monotonic()
degrees=2
E0=0
g1=0
g0=0
wp=40
El1=1e-3*wp
Ej=0.165*wp
Ec0=1.75*1e-4*wp*4
Ec1=0.378*wp*4
dim=np.asarray([71,400])
period=[0,0]
a=[-35,-10*pi]
b=[35,10*pi]
z=np.linspace(0,0,1)
ng=np.linspace(0,0,1)
#z=np.linspace(0,0,1)
#ng=np.linspace(0,1,101)
r=40
(B,fm2)=zero_pi(degrees,El1,Ej,Ec1,Ec0,E0,g1,g0,dim,period,a,b,z,ng,r)
print(time.monotonic()-start)


El1= 0.04
Ej= 6.6000000000000005
Ec1= 60.480000000000004
Ec0= 0.028
E0= 0
g1= 0
g0= 0


  0%|                                                                                            | 0/1 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [373]:
phi=np.linspace(a[1],b[1],dim[1])
U=El1*np.tensordot(theta**0,phi**2,axes=0)+2*Ej*(1-np.cos(np.tensordot(theta**0,phi,axes=0))*np.cos(np.tensordot(theta,phi**0,axes=0)))


In [5]:
def from_1d_2d(fm1,dim):
    f=tensordot(linspace(0,0,dim[0]),linspace(0,0,dim[1]),axes=0)
    for i in range(0,dim[0]):
        f[i,:]=fm1[i*dim[1]:(i+1)*dim[1]]
    return f

In [20]:
nq=r
f=[]
for i in range(0,nq):
    #plt.figure()
    f.append(from_1d_2d(fm2[:,i,0,0],dim))
    #plt.pcolormesh(f[i])

In [22]:
nq=40
f_200=[]
for i in range(0,nq):
    #plt.figure()
    f_200.append(from_1d_2d(fm2[:,i,0,0],dim))
    #plt.pcolormesh(f[i])

In [23]:
for i in range(0,40):
    if np.asarray(tf.einsum('ij,ij',f[i],f_200[i]))<0:
        f_200[i]=-f_200[i]
for i in range(0,40):
    print(tf.einsum('ij,ij',f[i],f_200[i]))

tf.Tensor(0.99999999999994, shape=(), dtype=float64)
tf.Tensor(0.99999999999994, shape=(), dtype=float64)
tf.Tensor(0.9999999999999343, shape=(), dtype=float64)
tf.Tensor(0.9999999999999121, shape=(), dtype=float64)
tf.Tensor(0.9999999999999286, shape=(), dtype=float64)
tf.Tensor(0.9999999999999368, shape=(), dtype=float64)
tf.Tensor(0.9999999999999367, shape=(), dtype=float64)
tf.Tensor(0.9999999999999281, shape=(), dtype=float64)
tf.Tensor(0.9999999999999217, shape=(), dtype=float64)
tf.Tensor(0.9999999999999474, shape=(), dtype=float64)
tf.Tensor(0.9999999999999327, shape=(), dtype=float64)
tf.Tensor(0.9999999999999535, shape=(), dtype=float64)
tf.Tensor(0.9999999999999475, shape=(), dtype=float64)
tf.Tensor(0.9999999999999587, shape=(), dtype=float64)
tf.Tensor(0.9999999999999556, shape=(), dtype=float64)
tf.Tensor(0.9999999999999638, shape=(), dtype=float64)
tf.Tensor(0.9999999999999656, shape=(), dtype=float64)
tf.Tensor(0.9999999999999806, shape=(), dtype=float64)
tf.Tensor(0.99

In [71]:
k=1
j=30
i=39
tf.einsum('ij,ij',f[i][k:-k][j:-j],f[i][k:-k][j:-j])

<tf.Tensor: shape=(), dtype=float64, numpy=0.061747618082262784>

In [189]:
plt.figure()
plt.pcolormesh(f[0])

In [195]:
f3=tf.einsum('ij,jk->ik',np.real(tf.linalg.expm(-1j*np.pi*np.diag(np.linspace(-35,35,71)))),f[0])

In [196]:
plt.figure()
plt.pcolormesh(f3)

In [204]:
abs(np.einsum('ij,ij',f3,f[1].conjugate()))**2

0.9029917994832402

In [197]:
n=np.linspace(a[0],b[0],dim[0])
theta=np.linspace(-np.pi/2,np.pi*3/2,251)
f4=n_phi(np.asarray(f3),n,theta)

In [198]:
plt.figure()
plt.pcolormesh(abs(f4))

In [140]:
ntheta=(np.diag(np.linspace(1,1,250),k=1)-np.diag(np.linspace(1,1,250),k=-1))
ntheta[0,-1]=-1
ntheta[-1,0]=1
f5=tf.einsum('ij,jk->ik',tf.linalg.expm(1/(theta[1]-theta[0])*ntheta*np.pi/2),f1[0])

In [114]:
np.gradient(f1[0],axis=0)
tf.einsum('ij,ij',tf.einsum('ij,ik->jk',ntheta,f1[0]),np.asarray(tf.einsum('ij,ik->jk',ntheta,f1[0])).conjugate())

<tf.Tensor: shape=(), dtype=float64, numpy=2.1193643086967477>

In [141]:
plt.figure()
plt.pcolormesh(abs(f5))

In [136]:
f5.shape

TensorShape([71, 401])

In [143]:
n=np.linspace(a[0],b[0],dim[0])
theta=np.linspace(-np.pi/2,np.pi*3/2,251)
f6=n_phi(np.asarray(f5),theta,n)

In [144]:
plt.figure()
plt.pcolormesh(abs(f6))

In [437]:
plt.figure()
plt.pcolormesh(abs(f1[1]))

In [422]:
x=np.linspace(-1/2,3/2,101)
a=np.sin(x*np.pi)

In [ ]:
np.roll(a,np.pi)

In [457]:
plt.figure()

plt.plot(x+1,a)
plt.plot(x,sdvig_pi(a))

ValueError: x and y must have same first dimension, but have shapes (101,) and (2,)

In [139]:
def n_phi(fm2,n,phi):
    f=tensordot(linspace(0j,0,phi.shape[0]),linspace(0j,0,fm2.shape[1]),axes=0)
    for j in range(0,fm2.shape[1]):
        for i in range(0,phi.shape[0]):
            f[i,j]=sum(np.exp(1j*phi[i]*n)*fm2[:,j])
    return f

In [140]:
f1=[]
for i in tqdm.tqdm(range(0,nq)):
    #plt.figure()
    n=np.linspace(a[0],b[0],dim[0])
    theta=np.linspace(-np.pi/2,np.pi*3/2,251)
    f1.append(n_phi(f[i],n,theta))
    #plt.pcolormesh(np.abs(f1[i]))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [05:32<00:00,  1.66s/it]


In [54]:
plt.figure()
plt.pcolormesh(np.abs(f2[0]))

In [143]:
def sdvig_pi(f):
    if f.shape[0]%2==0:
        print('ERROR Shape should be odd')
        return 0
    n=f.shape[0]//2
    f1=copy.copy(f)
    f1[:n+1]=f[n:]
    f1[n:]=f[:n+1]
    return f1

In [144]:
f2=[]
for i in range(0,1):
    plt.figure()
    f2.append(sdvig_pi(f1[i]))
    plt.pcolormesh(np.abs(f2[i]))

In [60]:
abs(np.einsum('ij,ij',f1[1],f2[0].conjugate())/(f1[i].shape[0]-1))**2

0.9029918688238342

In [61]:
c=[]
for i in range(0,nq):
    c.append(abs(np.einsum('ij,ij',f1[i],f2[0].conjugate())/(f1[i].shape[0]-1)))
sum(np.asarray(c)**2)

0.9030127374390042

In [9]:
np.asarray(f).shape

(40, 71, 351)

In [372]:
size=15
fig, axs = plt.subplots(ncols=3, nrows=2,figsize=(16,9),sharex=True)
phi=np.linspace(a[1],b[1],dim[1])
for i in range(0,2):
    axs[i,0].pcolormesh(phi,n,f[i])
    axs[i,0].set_ylabel(r'$n$',fontsize=size)
    axs[i,0].set_title(r'$\psi_{}$'.format(i),fontsize=size)
for i in range(0,2):
    axs[i,1].pcolormesh(phi,theta,abs(f1[i]))
    axs[i,1].set_ylabel(r'$\theta$',fontsize=size)
    axs[i,2].set_ylabel(r'$\theta$',fontsize=size)
    axs[i,1].set_title(r'$\psi_{}$'.format(i),fontsize=size)
axs[1,2].pcolormesh(phi,theta,abs(f2[0]))
axs[1,2].set_title(r'$\psi_{}$'.format(0),fontsize=size)
U=El1*np.tensordot(theta**0,phi**2,axes=0)+2*Ej*(1-np.cos(np.tensordot(theta**0,phi,axes=0))*np.cos(np.tensordot(theta,phi**0,axes=0)))
axs[0,2].pcolormesh(phi,theta,U)
axs[0,2].set_title(r'$U$',fontsize=size)
for i in range(0,3):
    axs[1,i].set_xlabel(r'$\phi$',fontsize=size)
plt.tight_layout()

TypeError: Incompatible X, Y inputs to pcolormesh; see help(pcolormesh)

## Оператор заряда

In [43]:
start=time.monotonic()
nq=r#dim[0]
n=np.diag(np.linspace(a[0]+0j,b[0],dim[0]))
#n_new=tf.einsum('mil,kil->mk',np.asarray(f),np.asarray(f).conjugate())
n_new=tf.einsum('mil,ij,kjl->mk',np.asarray(f),n,np.asarray(f).conjugate())
#n_new=np.round(n_new,6)
print(time.monotonic()-start)

0.06300000002374873


In [44]:
start=time.monotonic()
nq=40#dim[0]
n=np.diag(np.linspace(a[0]+0j,b[0],dim[0]))
#n_200=tf.einsum('mil,kil->mk',np.asarray(f_200),np.asarray(f_200).conjugate())
n_200=tf.einsum('mil,ij,kjl->mk',np.asarray(f_200),n,np.asarray(f_200).conjugate())
#n_200=np.round(n_200,6)
print(time.monotonic()-start)

0.046999999962281436


In [26]:
n=tf.einsum('mjl,kjl->mk',np.asarray(f),np.asarray(f_200).conjugate())

In [27]:
for i in range(0,40):
    print(i,abs(n[i])[np.argsort(-abs(n[i]))[1]])

0 tf.Tensor(4.4378913304418743e-10, shape=(), dtype=float64)
1 tf.Tensor(4.437930564822437e-10, shape=(), dtype=float64)
2 tf.Tensor(2.112696274738094e-10, shape=(), dtype=float64)
3 tf.Tensor(2.1127692604103128e-10, shape=(), dtype=float64)
4 tf.Tensor(5.57420823252214e-10, shape=(), dtype=float64)
5 tf.Tensor(5.574745907194089e-10, shape=(), dtype=float64)
6 tf.Tensor(1.2429507628181696e-11, shape=(), dtype=float64)
7 tf.Tensor(1.2431512618996721e-11, shape=(), dtype=float64)
8 tf.Tensor(3.031086799383411e-12, shape=(), dtype=float64)
9 tf.Tensor(7.244922826011092e-12, shape=(), dtype=float64)
10 tf.Tensor(7.256183981875884e-12, shape=(), dtype=float64)
11 tf.Tensor(4.3121701465272065e-12, shape=(), dtype=float64)
12 tf.Tensor(4.315335401200836e-12, shape=(), dtype=float64)
13 tf.Tensor(6.199736123935256e-10, shape=(), dtype=float64)
14 tf.Tensor(6.199628373729596e-10, shape=(), dtype=float64)
15 tf.Tensor(2.1020085473872394e-09, shape=(), dtype=float64)
16 tf.Tensor(2.10201320879785

In [45]:
np.linalg.norm(np.real(n_new-n_200))

0.08758566853051887

In [16]:
i=0
j=6
for i in range(0,40):
    print(i,(n_new-n_200)[35,i])

0 0j
1 0j
2 0j
3 0j
4 0j
5 0j
6 0j
7 0j
8 0j
9 0j
10 0j
11 0j
12 0j
13 0j
14 0j
15 0j
16 (8.000000000000001e-06+0j)
17 0j
18 0j
19 (-0+0j)
20 0j
21 0j
22 0j
23 0j
24 (0.0002509999999999999+0j)
25 0j
26 0j
27 0j
28 0j
29 (-0+0j)
30 0j
31 0j
32 0j
33 (-0+0j)
34 (-1.9999999999242846e-05+0j)
35 (-0.009829000000000001+0j)
36 0j
37 0j
38 0j
39 0j


In [295]:
nq=r
psi0=np.zeros((nq,1),dtype=complex)
psi0[0]=1
amp=(tf.linalg.expm(-1j*(n_new[:nq,:nq])*np.pi)@psi0)
abs(amp[1])**2

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.99999891])>

In [97]:
# think about writting of interaction hamiltoninian (there is an exponenta depending on time)
def evolution(nq,H1,V1,dt,step,w):
    start=time.monotonic()
    # density matrix
    #ro=np.zeros((nq,nq),dtype=complex)
    #ro[0,0]=1
    psi0=np.zeros((nq,1),dtype=complex)
    psi0[0]=1
    #psi0[1]=0
    #psi0=fm[:,0:1,0,0]
    psi=[]
    with tf.device("GPU:0"):
        for j in tqdm.tqdm(range(0,w.shape[0])):
            V=V1*w[j]
            psi1=[]
            psi1.append(psi0)
            for i in range(0,step):
                a=(tf.linalg.expm(-1j*(H1+V)*dt)@psi1[i])
                psi1.append(a/np.linalg.norm(a))
                
                #psi1.append(tf.linalg.expm(1j*(H1+V)*dt/w[j])@psi1[i])
                
                #psi1.append(np.linalg.norm(tf.linalg.expm(1j*(H1+V)*dt*i)@psi0))
                #psi1.append(tf.linalg.expm(1j*(H1+V)*dt*i)@psi0)
                #psi1.append(tf.linalg.expm(-1j*(V)*2*np.pi)@tf.linalg.expm(1j*(H1)*dt*i)@tf.linalg.expm(1j*(V)*np.pi)@psi0)
                #psi1.append(tf.linalg.expm(-1j*(V)*np.pi/2)@tf.linalg.expm(1j*(H1)*dt*i)@tf.linalg.expm(1j*(V)*np.pi/2)@psi0)
            psi.append(np.asarray(psi1))
        print(time.monotonic()-start)
    return np.asarray(psi)

In [183]:
w=np.linspace(3,12,9)
step=10000
dt=12.5/step
nq=40
(psi)=evolution(nq,B[:nq,0,0]-B[0,0,0],n_new[:nq,:nq],dt,step,w)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [23:00<00:00, 76.72s/it]

1380.8910000000033


In [192]:
max(np.abs(psi[0,:,1,0])**2)

0.9551772980790789

In [167]:
max(np.abs(psi[0,:,1,0])**2)

0.9123636707656496

In [55]:
psi.shape

(1, 2, 40, 1)

In [197]:
plt.figure()
step1=step+1
t=np.linspace(dt,step1*dt,step1)
plt.pcolormesh(t,w,np.abs(psi[:,:,1,0])**2)
plt.colorbar()

C:\Users\mazho\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:4: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  after removing the cwd from sys.path.


In [48]:
# only for digit w
def right_scale(w,dt,step,psi):
    plt.figure()
    for i in range(0,w.shape[0]):
        t=np.linspace(0,dt/w[i]*(step+1),(step+1))
        plt.pcolormesh(t,w[i:i+2],abs(psi[i:i+2,:])**2,vmin=0,vmax=0.9998341839370054)
        
    plt.colorbar()

In [33]:
right_scale(w,dt,step,psi[:,:,1,0])

C:\Users\mazho\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  


In [199]:
step1=step+1
t=np.linspace(dt,step1*dt,step1)
plt.figure()
for i in range(0,11):
    plt.plot(t,np.abs(psi[0,:step1,i,0])**2)